### This Reproducible Paper Project can be ran either in trovi terminal or jupyter notebook bash in reproduce.ipynb
### If you decide to run with the notebook, then dont forget to change the kernel to bash instead of python in the top right corner

#### Deactivate default conda env from trovi

In [1]:
conda init
conda deactivate

no change     /opt/conda/condabin/conda
no change     /opt/conda/bin/conda
no change     /opt/conda/bin/conda-env
no change     /opt/conda/bin/activate
no change     /opt/conda/bin/deactivate
no change     /opt/conda/etc/profile.d/conda.sh
no change     /opt/conda/etc/fish/conf.d/conda.fish
no change     /opt/conda/shell/condabin/Conda.psm1
no change     /opt/conda/shell/condabin/conda-hook.ps1
no change     /opt/conda/lib/python3.10/site-packages/xontrib/conda.xsh
no change     /opt/conda/etc/profile.d/conda.csh
modified      /home/radhofanazizi_gmail_com/.bashrc

==> For changes to take effect, close and re-open your current shell. <==



#### For `check.py` and `probfuzz.py`, replace the first shebang line with:
```
#!/usr/bin/python2
```

#### For install.sh file, replace last line with:
```
chmod +x check.py
python2 ./check.py
```

#### Install dependencies

In [2]:
sudo ./install.sh

Get:1 http://archive.ubuntu.com/ubuntu jammy InRelease [270 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy/main amd64 Packages [1,792 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy/multiverse amd64 Packages [266 kB]
Get:7 http://archive.ubuntu.com/ubuntu jammy/restricted amd64 Packages [164 kB]
Get:8 http://archive.ubuntu.com/ubuntu jammy/universe amd64 Packages [17.5 MB]
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [2,912 kB]
Get:10 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,526 kB]
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [3,748 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,610 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-upda

#### Install antler 4.7.1 inside trovi again since it gives error using the normal install.sh

In [3]:
pip2 install antlr4-python2-runtime==4.7.1

DEPRECATION: Python 2.7 reached the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 is no longer maintained. pip 21.0 will drop support for Python 2.7 in January 2021. More details about Python 2 support in pip can be found at https://pip.pypa.io/en/latest/development/release-process/#python-2-support pip 21.0 will remove support for this functionality.
     |████████████████████████████████| 111 kB 1.0 MB/s eta 0:00:01
  Created wheel for antlr4-python2-runtime: filename=antlr4_python2_runtime-4.7.1-py2-none-any.whl size=140332 sha256=d9618da071b00c5ee68b3409d896d77b505a3e80605a88bb710a7cac3ac8699e
  Stored in directory: /home/radhofanazizi_gmail_com/.cache/pip/wheels/92/e9/cd/cba2a81b5ce7fb940ea123148c0345c1afbe70ffa48d1c3e97
Successfully built antlr4-python2-runtime


#### Run probfuzz:  

In [ ]:
python2 ./probfuzz.py 5